In [10]:
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import pandas as pd
import requests
import bs4
import re
from tqdm import tqdm

# 爬ettoday新聞網
def craw_ettoday(hours=2): # hours=2控制資料量
    browser = webdriver.Chrome(executable_path='./chromedriver')
    browser.get("https://www.ettoday.net/news/news-list.htm")
    two_hours_ago = datetime.now() - timedelta(hours = hours)
    print(two_hours_ago)
    two_hours_ago_time = two_hours_ago.strftime("%Y/%m/%d %H:%M")
    print("兩小時前時間：", two_hours_ago_time)
    last_height = browser.execute_script("return document.body.scrollHeight")
    go=True
    while go:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        html_source = browser.page_source
        soup = BeautifulSoup(html_source, "lxml")
        new_height = browser.execute_script("return document.body.scrollHeight")
        #已經到頁面底部
        if new_height == last_height:
            print("已經到頁面最底部，程序停止")
            break
        last_height = new_height
        for d in soup.find(class_="part_list_2").find_all('h3'):
            #已經超出兩小時
            if datetime.strptime(d.find(class_="date").text, '%Y/%m/%d %H:%M') < two_hours_ago:
                print("已經超出兩個小時，程序停止")
                go = False
                break
            else:
                print("目前畫面最下方文章的日期時間為：",d.find_all(class_="date")[-1].text)
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "lxml")
    news ={"日期時間":[],"標題":[],"連結":[]}
    for d in tqdm(soup.find(class_="part_list_2").find_all('h3')):
        if two_hours_ago_time in d.find(class_="date") :
            pass
        else:
            print(d.find(class_="date").text, d.find_all('a')[-1].text)
            news["日期時間"].append(d.find(class_="date").text)
            news["標題"].append(d.find_all('a')[-1].text)
            news["連結"].append("https://www.ettoday.net" + d.find_all('a')[-1]["href"])
    browser.close()  #很重要喔記得要關掉瀏覽器．不然要手動關掉
    df_news=pd.DataFrame(news)
    df_content = pd.DataFrame(columns=['title','url','content'])
    for idx in df_news.index:
        標題 = df_news.loc[idx,'標題']
        連結 = df_news.loc[idx,'連結']
        內文  = bs4.BeautifulSoup(requests.get(連結).text,'html.parser').find_all('div',class_='story')[0].text.replace('\n','')
        # 一個dataframe 三個欄位
        df_content.loc[idx,'title'] = 標題
        df_content.loc[idx,'url'] = 連結
        df_content.loc[idx,'content'] = 內文
    return df_content

In [11]:
res = craw_ettoday()
res

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10476\2935942173.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path='./chromedriver')


2023-02-01 08:28:10.784765
兩小時前時間： 2023/02/01 08:28
已經到頁面最底部，程序停止


100%|██████████| 100/100 [00:00<00:00, 10028.46it/s]

2023/02/01 10:26 薔薔日本扮藝妓「被各種名目A錢」怒了：很坑人！店家被狂洗版  慘剩2顆星
2023/02/01 10:26 陳美鳳日本行名牌帽出籠　與林俊傑隔海撞LV夯款
2023/02/01 10:23 女駕駛交流道追撞前車　「邊講電話邊開門」又A到保時捷
2023/02/01 10:22 免會員！超夯熱賣商品「優惠1次看」　洗衣精、衛生紙、花月嵐
2023/02/01 10:21 國台辦批蔡英文通話捷克準總統　陸委會：中華民國有權與世界往來
2023/02/01 10:19 郭台銘有意組「非綠陣營」拚2024？　侯友宜：只要台灣好都全力以赴
2023/02/01 10:18 牡羊座生日禮！柯建銘：普發6000元「4月第二周三讀」
2023/02/01 10:18 名校地下埋藏巨大機密被發現！瘋傳「秘建數十層」　哈工程回應
2023/02/01 10:13 助小熊隊打破「山羊魔咒」　奪冠功臣佛勒宣布退休
2023/02/01 10:13 洗衣機壞掉「要換洗脫烘」　人夫怕爆了！網勸買這1物：老婆愛死你
2023/02/01 10:12 免費早餐吃到爽！住星級飯店「每人只要1250元」　再送樂園門票
2023/02/01 10:12 基隆市民過半支持升格直轄市　謝國樑：反映民意...3月拜會林右昌
2023/02/01 10:11 徐巧芯稱2020選得「風雨飄搖」　費鴻泰甚至在「這區」輸了
2023/02/01 10:09 今年是「雙二月」必拜土地公！2月6日最旺　招財旺桃花「拜法完整攻略」
2023/02/01 10:07 休旅車囂張插人行道！他檢舉卻被說「違規樣態不明顯」　警解釋了
2023/02/01 10:03 最後身影曝！苗栗婦失蹤前曾偶遇好友　8天後尋獲…離奇陳屍池塘
2023/02/01 10:03 必勝客買1送1、漢堡王7折吃！　foodpanda「2月激省優惠碼」一次看
2023/02/01 10:00 雞塊加1元多1件、咖啡買1送1！　麥當勞「超殺優惠券」爽吃42天
2023/02/01 10:00 高攀得起！天母也有4字頭房　在地人駁鄉民刻板印象
2023/02/01 10:00 南方澳美食推這家！划算鮮魚湯69元起　綿密魚蛋沙拉每桌必點
2023/02/01 09:59 快訊／國民黨喊加碼發現金1萬　陳建仁：應照顧弱勢團體
2

,title,url,content
0,薔薔日本扮藝妓「被各種名目A錢」怒了：很坑人！店家被狂洗版 慘剩2顆星,https://www.ettoday.net/news/20230201/2431248.htm,記者潘慧中／綜合報導薔薔（林嘉凌）最近飛往日本遊玩時，帶朋友一起體驗花魁妝扮，「以為是平靜的...
1,陳美鳳日本行名牌帽出籠 與林俊傑隔海撞LV夯款,https://www.ettoday.net/news/20230201/2431034.htm,▲陳美鳳戴著LV草間彌生聯名漁夫帽，與藍心湄在日本享美食。（圖／翻攝陳美鳳臉書）記者陳雅韻／...
2,女駕駛交流道追撞前車 「邊講電話邊開門」又A到保時捷,https://www.ettoday.net/news/20230201/2431303.htm,▲國道女三寶處理完追撞，下秒又碰撞保時捷。（圖／翻攝自記者爆料網）記者許宥孺／高雄報導國道1...
3,免會員！超夯熱賣商品「優惠1次看」 洗衣精、衛生紙、花月嵐,https://www.ettoday.net/news/20230201/2413470.htm,文／壹五零物價上漲，精打細算買到最優惠的商品，是件重要的事情，本文幫大家整理出，超夯的熱賣商...
4,國台辦批蔡英文通話捷克準總統 陸委會：中華民國有權與世界往來,https://www.ettoday.net/news/20230201/2431318.htm,▲總統蔡英文與捷克總統當選人帕維爾通話。（圖／總統府）記者蔡紹堅／綜合報導總統蔡英文日前與捷...
...,...,...,...
95,美卿布林肯下周訪中2天 將與北京官員討論俄烏戰爭,https://www.ettoday.net/news/20230201/2431191.htm,▲布林肯訪中受矚目。（圖／路透）記者張靖榕／編譯美國白宮國安會戰略溝通協調官柯比（John ...
96,雙寶媽突告白「LINE置頂你」 醫：別害我睡公園,https://www.ettoday.net/news/20230201/2431187.htm,▲雙寶媽一席話，讓醫師當下不知該如何回應。（示意圖／翻攝pixabay免費圖庫）記者陳俊宏／...
97,她失戀看A片看到哭 一票妹子湧入安慰「我也是」！哭點在這,https://www.ettoday.net/news/20230201/2431189.htm,▲女子看A片想起前男友，忍不住大哭。（示意圖／取自免費圖庫Pixabay）記者崔子柔／綜合報...
98,荷蘭籲境內半導體商加把勁 防晶片最終落入俄手中,https://www.ettoday.net/news/20230201/2431190.htm,▲荷蘭半導體製造商生產的晶片，被大量流入俄羅斯手中。（圖／路透）文／中央社阿姆斯特丹31日綜...


In [13]:
display(res.head())

,title,url,content
0,薔薔日本扮藝妓「被各種名目A錢」怒了：很坑人！店家被狂洗版 慘剩2顆星,https://www.ettoday.net/news/20230201/2431248.htm,記者潘慧中／綜合報導薔薔（林嘉凌）最近飛往日本遊玩時，帶朋友一起體驗花魁妝扮，「以為是平靜的...
1,陳美鳳日本行名牌帽出籠 與林俊傑隔海撞LV夯款,https://www.ettoday.net/news/20230201/2431034.htm,▲陳美鳳戴著LV草間彌生聯名漁夫帽，與藍心湄在日本享美食。（圖／翻攝陳美鳳臉書）記者陳雅韻／...
2,女駕駛交流道追撞前車 「邊講電話邊開門」又A到保時捷,https://www.ettoday.net/news/20230201/2431303.htm,▲國道女三寶處理完追撞，下秒又碰撞保時捷。（圖／翻攝自記者爆料網）記者許宥孺／高雄報導國道1...
3,免會員！超夯熱賣商品「優惠1次看」 洗衣精、衛生紙、花月嵐,https://www.ettoday.net/news/20230201/2413470.htm,文／壹五零物價上漲，精打細算買到最優惠的商品，是件重要的事情，本文幫大家整理出，超夯的熱賣商...
4,國台辦批蔡英文通話捷克準總統 陸委會：中華民國有權與世界往來,https://www.ettoday.net/news/20230201/2431318.htm,▲總統蔡英文與捷克總統當選人帕維爾通話。（圖／總統府）記者蔡紹堅／綜合報導總統蔡英文日前與捷...
